# RATRAN benchmark: example 0

## 0) Setup

In [1]:
# Define Magritte folders

MagritteSetupFolder = '/home/frederik/Dropbox/Astro/Magritte/modules/setup/'
ProjectFolder       = '/home/frederik/MagritteProjects/Ratran/'

In [2]:
# Import tools and libraries

import numpy as np

from bokeh.plotting import figure, show, gridplot
from bokeh.palettes import cividis
from bokeh.io       import output_notebook
output_notebook()

from glob import glob
from sys  import path
path.insert(0, MagritteSetupFolder)

# Import from MagritteSetupFolder
from spheres  import deSphere, sphereVar, nRays
from lineData import LineData, planck, relativeDifference
from setup    import setupMagritte
from model    import model, mapToXD

Loading BokehJS ...

## 1) Define model

Assume a 1D model (i.e. 1 ray) with constant temperature, density and abundances and a linear velocity field, yielding a constant velocity gradient

\begin{align}
T(r)     \ &= \ cte, \\
\rho(r)  \ &= \ cte, \\
n_{i}(r) \ &= \ cte, \\
v(r)     \ &= \ \frac{v_{\max} r}{L},
\end{align}

where $L$ is the total length of the model and $v_{\max}$ is the maximum speed reached in the model. (Numerical values given below.) We write it in this way to ensure that velocities in the model are always much smaller than $c$, since our equations for Doppler shift are not valid otherwise.

There is CMB radiation incoming on both ends of the ray as boundary condition.

In [3]:
# Read Ratran data
folder = '/home/frederik/Codes/Ratran/example/frederik/'
temperatureR           = np.loadtxt(folder + 'temperature.txt')
(densityR, abundanceR) = np.loadtxt(folder + 'abundance.txt', unpack=True)
(R, VR)                = np.loadtxt(folder + 'grid.txt',      unpack=True)

model = model (dim=1)

model.ncells = len(R)

model.density     = densityR
model.abundance   = abundanceR
model.temperature = temperatureR

model.x = R
model.y = [0.0 for _ in range(model.ncells)]
model.z = [0.0 for _ in range(model.ncells)]

model.vx = VR
model.vy = [0.0 for _ in range(model.ncells)]
model.vz = [0.0 for _ in range(model.ncells)]

model.boundary = [0, model.ncells-1]

model.defineRays (nrays=2)

model.getNeighborLists ()

# Write new model data
model.writeInput (ProjectFolder + 'Magritte_files/')

# Run setup
setupMagritte (projectFolder = ProjectFolder, runName = '')

print(f'ncells = {model.ncells}')
print(f'nrays  = {model.nrays} ')

ncells = 16
nrays  = 2 


In [4]:
# Plot model
plot_model_1 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_1.line (model.x, model.density)
plot_model_1.xaxis.axis_label = "x [m]"
plot_model_1.yaxis.axis_label = "density [m^-3]"

plot_model_2 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_2.line (model.x, model.abundance)
plot_model_2.xaxis.axis_label = "x [m]"
plot_model_2.yaxis.axis_label = "abundance [xm^-3]"

plot_model_3 = figure(plot_width=400, plot_height=250)
plot_model_3.line (model.x, model.temperature)
plot_model_3.xaxis.axis_label = "x [m]"
plot_model_3.yaxis.axis_label = "temperature [K]"

plot_model_4 = figure(plot_width=400, plot_height=250)
plot_model_4.line (model.x, model.vx)
plot_model_4.xaxis.axis_label = "x [m]"
plot_model_4.yaxis.axis_label = "velocity_x [m^-3]"

plot_model = gridplot ([[plot_model_1, plot_model_2],[plot_model_3, plot_model_4]])

show (plot_model)

### Get Magritte output

In [7]:
ioFolders = glob(ProjectFolder + 'io/*/')
ioFolders.sort()

inputFolders  = [folder +  'input/' for folder in ioFolders]
outputFolders = [folder + 'output/' for folder in ioFolders]

lastOutput = outputFolders[-1]
lastInput  =  inputFolders[-1]

print(lastOutput)

pops_files = glob(lastOutput + 'populations_0*.txt')
Jeff_files = glob(lastOutput + 'Jeff_0*.txt')
J_files    = glob(lastOutput + 'J_*.txt')
G_files    = glob(lastOutput + 'G_*.txt')
nu_files   = glob(lastOutput + 'frequencies_nu*.txt')
lnr_files  = glob(lastOutput + 'frequencies_line_nr*.txt')
eta_files  = glob(lastOutput + 'eta_0*.txt')
chi_files  = glob(lastOutput + 'chi_0*.txt')

pops_files.sort()
Jeff_files.sort()
eta_files.sort()
chi_files.sort()

pops_data = [np.loadtxt(fileName) for fileName in pops_files]
Jeff_data = [np.loadtxt(fileName) for fileName in Jeff_files]
J_data    = [np.loadtxt(fileName) for fileName in J_files]
G_data    = [np.loadtxt(fileName) for fileName in G_files]
nu_data   = [np.loadtxt(fileName) for fileName in nu_files]
lnr_data  = [np.loadtxt(fileName) for fileName in lnr_files]
eta_data  = [np.loadtxt(fileName) for fileName in eta_files]
chi_data  = [np.loadtxt(fileName) for fileName in chi_files]

# Import linedata
lineData = LineData (ProjectFolder + '/Magritte_files/linedata/hco+.dat')

/home/frederik/MagritteProjects/Ratran/io/18-12-03_21:30:41/output/


### Plot output

In [6]:
# Plot functions

def color(s):
    ns = int((s_max-s_min) / s_step + 1)
    es = int((s    -s_min) / s_step)
    return cividis(ns)[es]

def legend(s):
    return f'{s}'

def bokeh_log_plot(title, x, y, xlabel, ylabel):
    return

In [7]:
s_min  = 0
s_max  = model.ncells
s_step = 1

In [8]:
# Level populations

plot = figure (title='Level populations', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = range(lineData.nlev)
    y = pops_data[0][s]
    plot.line (x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "number of the level"
plot.yaxis.axis_label = "population [m^-3]"
show (plot)

In [9]:
# Mean intensity

plot = figure (title='Total mean intensity', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = range(lineData.nrad)
    y = Jeff_data[1][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "number of the transition"
plot.yaxis.axis_label = "mean intensity J [m^-3]"
show(plot)

In [10]:
# Spectrum

plot = figure (title='Spectrum', width=700, height=500, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = nu_data[0][s]
    y =  J_data[0][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

In [11]:
# Flux (G) spectrum

plot = figure (title='Spectrum', width=700, height=500)
for s in range(s_min, s_max, s_step):
    x = nu_data[0][s]
    y =  G_data[0][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

## Analytical solution

In [8]:
line = 5

In [9]:
c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg]  proton mass
T_CMB = 2.7254800        # [K]   CMB temperature
vturb = 0.12012E3        # [m/s] turbulent speed

L    = model.x[-1]
vmax = model.vx[-1]
nuij = lineData.frequency[line]


def TT(x):
    #return efunc(x, c_temp)
    if   (x <= model.x[0]):
        return model.temperature[0]
    elif (x >= model.x[-1]):
        return model.temperature[-1]
    else:
        return interpolate(model.x, model.temperature)(x)

def VV(x):
    #return -efunc(x, c_velo)
    if   (x <= model.x[0]):
        return model.vx[0]
    elif (x >= model.x[-1]):
        return model.vx[-1]
    else:
        return interpolate(model.x, model.vx)(x)

def abn(x):
    #return efunc(x, c_abun)
    if   (x <= model.x[0]):
        return model.abundance[0]
    elif (x >= model.x[-1]):
        return model.abundance[-1]
    else:
        return interpolate(model.x, model.abundance)(x)


from scipy.integrate   import quad      as integrate
from scipy.interpolate import interp1d  as interpolate

def pops(x):
    return lineData.LTEpop(TT(x)) * abn(x)

def emissivity(x):
    return lineData.lineEmissivity(pops(x))

def opacity(x):
    return lineData.lineOpacity(pops(x))

def source(T):
    return emissivity(x) / opacity(x)

def B(nu):
    return planck(T_CMB, nu)

def dnu(x):
    return nuij/c * np.sqrt(2.0*kb*TT(x)/mp + vturb**2)

def phi(nu, x):
    return 1 / (np.sqrt(np.pi) * dnu(x)) * np.exp(-((nu-nuij)/dnu(x))**2)

def eta(nu, x):
    return emissivity(x)[line] * phi(nu, x)

def chi(nu, x):
    return    opacity(x)[line] * phi(nu, x)

def doppler(x1,x2):
    return 1 - (VV(x2) - VV(x1)) / c

def tau_p(nu, x, xs):
    return integrate (lambda xp: chi(nu*doppler(xp,x), xp), xs, x)[0]

Assuming boundary condition $B_{\nu}$ on both sides of the ray, the intensity is given by

\begin{align}
I^{+}_{\nu}(x) \ &= \ B_{\nu} e^{-\tau_{\nu'}(0,x)} + \int_{0}^{x} \text{d}x' \ \eta_{\nu'}(x') \ e^{-\tau_{\nu'}(x', x)} \\
I^{-}_{\nu}(x) \ &= \ B_{\nu} e^{-\tau_{\nu'}(x,L)} + \int_{x}^{L} \text{d}x' \ \eta_{\nu'}(x') \ e^{-\tau_{\nu'}(x, x')}
\end{align}

where the optical depth $\tau^{\pm}_{\nu}$ is defined

\begin{equation}
    \tau_{\nu}(x_{1}, x_{2}) \ = \ \int_{x_{1}}^{x_{2}} \text{d} x' \ \chi_{\nu}(x') .
\end{equation}

The mean intensity is hence given by

\begin{equation}
    J_{\nu}(x) \ = \ \frac{1}{2} \big( I^{+}_{\nu}(x) \ + \ I^{-}_{\nu}(x) \big)
\end{equation}

The frequency dependence of the emissivity and opacity only comes from the line profile

\begin{align}
    \eta_{\nu}(x) \ &= \ \eta_{ij} \phi_{\nu}, \\
    \chi_{\nu}(x) \ &= \ \chi_{ij} \phi_{\nu},
\end{align}

where we assume a Gaussian profile

\begin{equation}
	\phi_{\nu}^{ij}(x) \ = \ \frac{1}{\sqrt{\pi} \ \delta\nu_{ij}(x)} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x)}\right)^{2}\right], \hspace{5mm} \text{where} \hspace{5mm} \delta\nu_{ij}(x) \ = \ \frac{\nu_{ij}}{c} \sqrt{ \frac{2 k_{b} T(x)}{m_{\text{spec}}} \ + \ v_{\text{turb}}^{2}(x)}.
\end{equation}

Solving the integral for the optical depth then yields

\begin{equation}
  \tau^{\pm}_{\nu}(x) \ = \ \frac{\chi}{\sqrt{\pi}} \ \int_{0}^{\ell} \text{d} l \ \frac{1}{\delta\nu_{ij}(x \pm l)} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x \pm l)}\right)^{2}\right] .
\end{equation}

In [10]:
from os  import getcwd
from sys import path
path.insert(0, f'{getcwd()}/functions')

from functions import test, optical_depth

In [18]:
nu  = nuij
chi = opacity(model.x[1])[line]

start = model.x[0]
stop  = model.x[5]

dx = (stop-start) / 1000

print(optical_depth (nu, chi, nuij, start, stop, dx))
print(tau_p(nu, model.x[5], model.x[0]))

2.784506357481779e-10


TypeError: 'numpy.float64' object is not callable

In [9]:
from scipy.optimize    import curve_fit as fit
from scipy.interpolate import interp1d  as interpolate

def func(x, a, b, c):
    return a + 1.0 / x**c

def efunc(x, coeffs):
    return func(x, coeffs[0], coeffs[1], coeffs[2])

#lx    = np.insert(model.x          , 0, 0.9*model.x[0])
#ltemp = np.insert(model.temperature, 0, 10.0)
#lvelo = np.insert(model.vx         , 0, -3000.0)
#labun = np.insert(model.abundance  , 0, 0.0)

lx    = model.x
ltemp = model.temperature
lvelo = model.vx
labun = model.abundance

# lx    = np.log(np.abs(lx   ) + 1.0)
# ltemp = np.log(np.abs(ltemp) + 1.0)
# lvelo = np.log(np.abs(lvelo) + 1.0)
# labun = np.log(np.abs(labun) + 1.0)

# ilx    = np.linspace(lx[0], lx[-1], 1000)
# iltemp = interpolate(lx, ltemp)(ilx)
# ilvelo = interpolate(lx, lvelo)(ilx)
# ilabun = interpolate(lx, labun)(ilx)

(c_temp, cov_temp) = fit (func, lx, ltemp)
(c_velo, cov_velo) = fit (func, lx, lvelo)
(c_abun, cov_abun) = fit (func, lx, labun)

def TT(x):
    #return efunc(x, c_temp)
    if   (x <= model.x[0]):
        return model.temperature[0]
    elif (x >= model.x[-1]):
        return model.temperature[-1]
    else:
        return interpolate(model.x, model.temperature)(x)

def VV(x):
    #return -efunc(x, c_velo)
    if   (x <= model.x[0]):
        return model.vx[0]
    elif (x >= model.x[-1]):
        return model.vx[-1]
    else:
        return interpolate(model.x, model.vx)(x)

def abn(x):
    #return efunc(x, c_abun)
    if   (x <= model.x[0]):
        return model.abundance[0]
    elif (x >= model.x[-1]):
        return model.abundance[-1]
    else:
        return interpolate(model.x, model.abundance)(x)

c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg]  proton mass
T_CMB = 2.7254800        # [K]   CMB temperature
vturb = 0.12012E3        # [m/s] turbulent speed

line = 5

L    = model.x[-1]
vmax = model.vx[-1]
nuij = lineData.frequency[line]

xlin = np.linspace(model.x[0], model.x[-1], 1000)

/home/frederik/software/anaconda3/envs/magritte/lib/python3.6/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


NameError: name 'lineData' is not defined

In [48]:
print(ltemp)

[115.57   81.791  72.044  63.46   55.898  49.237  43.37   38.202  33.65
  29.64   26.108  22.997  20.257  17.843  15.717  13.844]


In [50]:
# Plot model
plot_model_1 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_1.line (model.x, model.density)
plot_model_1.xaxis.axis_label = "x [m]"
plot_model_1.yaxis.axis_label = "density [m^-3]"

plot_model_2 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_2.line   (model.x, model.abundance)
plot_model_2.circle (xlin, abn(xlin))
plot_model_2.xaxis.axis_label = "x [m]"
plot_model_2.yaxis.axis_label = "abundance [xm^-3]"

plot_model_3 = figure(plot_width=400, plot_height=250)
plot_model_3.line (model.x, model.temperature)
plot_model_3.circle (xlin, TT(xlin))
plot_model_3.xaxis.axis_label = "x [m]"
plot_model_3.yaxis.axis_label = "temperature [K]"

plot_model_4 = figure(plot_width=400, plot_height=250)
plot_model_4.line (model.x, model.vx)
plot_model_4.circle (xlin, VV(xlin))
plot_model_4.xaxis.axis_label = "x [m]"
plot_model_4.yaxis.axis_label = "velocity_x [m^-3]"

plot_model = gridplot ([[plot_model_1, plot_model_2],[plot_model_3, plot_model_4]])

show (plot_model)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [17]:
from scipy.integrate import quad as integrate

def pops(x):
    return lineData.LTEpop(TT(x)) * abn(x)

def emissivity(x):
    return lineData.lineEmissivity(pops(x))

def opacity(x):
    return lineData.lineOpacity(pops(x))

def source(T):
    return emissivity(x) / opacity(x)

def B(nu):
    return planck(T_CMB, nu)

def dnu(x):
    return nuij/c * np.sqrt(2.0*kb*TT(x)/mp + vturb**2)

def phi(nu, x):
    return 1 / (np.sqrt(np.pi) * dnu(x)) * np.exp(-((nu-nuij)/dnu(x))**2)

def eta(nu, x):
    return emissivity(x)[line] * phi(nu, x)

def chi(nu, x):
    return    opacity(x)[line] * phi(nu, x)

def doppler(x1,x2):
    return 1 - (VV(x2) - VV(x1)) / c

def tau_p(nu, x, xs):
    return integrate (lambda xp: chi(nu*doppler(xp,x), xp), xs, x)[0]

def tau_m(nu, x, xs):
    return integrate (lambda xp: chi(nu*doppler(x,xp), xp), x, xs)[0]

def I_p(nu, x):
    result  = B(nu) * np.exp(-tau_p(nu, 0, x))
    result += integrate (lambda xp: eta(nu*doppler(xp,x), xp) * np.exp(-tau_p(nu, x, xp)), 0, x)[0]
    return result

def I_m(nu, x):
    result  = B(nu) * np.exp(-tau_m(nu, x, L))
    result += integrate (lambda xp: eta(nu*doppler(x,xp), xp) * np.exp(-tau_m(nu, x, xp)), x, L)[0]
    return result
    
def J(nu, x):
    return 0.5 * (I_p(nu, x) + I_m(nu, x))

def relativeError(a,b):
    return 2.0 * np.abs((a-b)/(a+b))

In [ ]:


tau_m (nu, x, xs)

In [53]:
# Line

plot_model = figure(title='Line model', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    # model
    x = nuij + 15 * dnu(model.x[s]) * np.linspace(-1,1,100)
    y = [J(xi, model.x[s]) for xi in x]
    plot_model.line(x, y, color=color(s))
    # data
    x = nu_data[0][s][M:N]
    y =  J_data[0][s][M:N]
    plot_model.circle(x, y, color=color(s), legend=legend(s))

# plot_error = figure(title='Line error', width=400, height=400, y_axis_type="log")
# for s in range(s_min, s_max, s_step):
#     M = int(lnr_data[0][s][line] - 18    )
#     N = int(lnr_data[0][s][line] + 18 + 1)
#     # error
#     x  = nu_data[0][s][M:N]
#     JJ = [J(xi, model.x[s]) for xi in x]
#     y = relativeError(JJ, J_data[0][s][M:N])
#     plot_error.circle(x, y, color=color(s), legend=legend(s))

plot = gridplot([[plot_model]])

show(plot)

/home/frederik/software/anaconda3/envs/magritte/lib/python3.6/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)
/home/frederik/software/anaconda3/envs/magritte/lib/python3.6/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


KeyboardInterrupt: 

In [25]:
plot_line_model = figure(title='Line model: space dependence', width=450, height=400, y_axis_type="log")
#for s in range(s_min, s_max, 2):
# model
xx = np.array(model.x)
plot_line_model.line(xx, J(nuij,xx))
# data
M = int(lnr_data[0][0][line])
plot_line_model.circle(xx, J_data[0][:,M])
plot_line_model.xaxis.axis_label = "x [m]"
plot_line_model.yaxis.axis_label = "Mean intensity [W/m^2]"

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)

In [26]:
plot_line_model = figure(title='Line model', width=450, height=400, y_axis_type="log")
#for s in range(s_min, s_max, 2):
# model
xx = np.array(model.x)
plot_line_model.line(xx, J(nuij,xx))
# data
M = int(lnr_data[0][0][line])
plot_line_model.circle(xx, Jeff_data[0][:,line])

plot_line = gridplot([[plot_line_model, plot_line_error]])

show (plot_line)